עמית טיקוצינסקי - 206963985
קובי קראדי - 208748566

https://github.com/amittiko/cars_prediction.git

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


dataset = pd.read_csv('dataset.csv')


In [6]:
import pandas as pd
from datetime import datetime
import re

def prepare_data(dataset):
    # Ensure we are working with a copy to avoid warnings
    dataset = dataset.copy()
    
    # Drop rows with NaN in specific columns
    dataset = dataset.dropna(subset=["Gear"])
    dataset = dataset.dropna(subset=["capacity_Engine"])
    dataset = dataset.dropna(subset=["Engine_type"])

    # Fill NaN values in 'Color' and replace specific values
    dataset['Color'] = dataset['Color'].fillna('חסר צבע')
    dataset['Color'] = dataset['Color'].replace("None", 'חסר צבע')
    
    # המיפוי לצבעים מאוחדים
    color_mapping = {
        'כחול כהה מטאלי': 'כחול',
        'כחול בהיר': 'כחול',
        'אפור מטאלי': 'אפור',
        'חסר צבע': 'חסר צבע',
        'שחור': 'שחור',
        'חום': 'חום',
        'כסוף': 'כסף',
        'לבן': 'לבן',
        'לבן מטאלי': 'לבן',
        'לבן פנינה': 'לבן',
        'אפור עכבר': 'אפור',
        'אפור': 'אפור',
        'כחול': 'כחול',
        'סגול': 'סגול',
        'אדום': 'אדום',
        'כסף מטלי': 'כסף',
        'כתום': 'כתום',
        'לבן שנהב': 'לבן',
        'סגול חציל': 'סגול',
        'כסוף מטאלי': 'כסף',
        'כחול בהיר מטאלי': 'כחול',
        'טורקיז': 'טורקיז',
        "בז'": 'בז',
        'בורדו': 'בורדו',
        'ירוק': 'ירוק',
        'שמפניה': 'לבן',
        'ירוק מטאלי': 'ירוק',
        'תכלת': 'תכלת',
        'חום מטאלי': 'חום',
        'אדום מטאלי': 'אדום',
        'כחול מטאלי': 'כחול',
        "בז' מטאלי": 'בז',
        'ורוד': 'ורוד',
        'ברונזה': 'ברונזה',
        'ירוק בהיר': 'ירוק',
        'זהב מטאלי': 'זהב',
        'תכלת מטאלי': 'תכלת',
        'זהב': 'זהב'
    }

    # פונקציה להחלפת הערכים על פי המיפוי
    def map_colors(dataset):
        dataset['Color'] = dataset['Color'].map(color_mapping).fillna(dataset['Color'])
        return dataset

    # להחיל את הפונקציה על הדאטה סט שלך
    dataset = map_colors(dataset)

    # Correcting specific values
    replacements = {
        'manufactor': {"Lexsus": 'לקסוס'},
        'Gear': {"אוטומט": 'אוטומטית'},
        'Engine_type': {"היבריד": 'היברידי', "טורבו דיזל": 'דיזל'},
        'Prev_ownership': {"אחר": 'לא מוגדר', "None": 'לא מוגדר'},
        'Curr_ownership': {"אחר": 'לא מוגדר', "None": 'לא מוגדר'},
        'City': {
            "ashdod": 'אשדוד', "Tel aviv": 'תל אביב יפו', "Tzur Natan": 'צור נתן',
            "haifa": 'חיפה', "Rehovot": 'רחובות', "jeruslem": 'ירושלים', 
            "Rishon LeTsiyon": 'ראשון לציון', 'פ"ת': 'פתח תקווה', 
            'פתח': 'פתח תקווה', "תל": 'תל אביב יפו', "תל אביב": 'תל אביב יפו', 
            "מזכרת": 'מזכרת בתיה', "ק.אתא": 'קריית אתא', "באקה": 'באקה אל גרבייה', 
            "חיפ": 'חיפה'
        }
    }
    for column, replace_dict in replacements.items():
        dataset[column] = dataset[column].replace(replace_dict)
    
    dataset = dataset.query('Gear != "לא מוגדר"')
    dataset['capacity_Engine'] = dataset['capacity_Engine'].str.replace(',', '', regex=True)

    # Fill NaN values in 'Prev_ownership' and 'Curr_ownership'
    dataset['Prev_ownership'] = dataset['Prev_ownership'].fillna('לא מוגדר')
    dataset['Curr_ownership'] = dataset['Curr_ownership'].fillna('לא מוגדר')

    # Filter out specific cities
    dataset = dataset.query('City not in ["ראש", "הוד", "פתח תקווה,יהוד", "חד", "אבן", "כפר", "בת", "רא", "רמת","קרית","אומן"]')

    # Fill NaN values in 'Pic_num' with 0
    dataset['Pic_num'] = dataset['Pic_num'].fillna(0)

    # Remove rows where 'Km' is "None"
    dataset = dataset.query('Km != "None"')
    dataset['Km'] = dataset['Km'].str.replace(',', '', regex=True)

    # Convert 'Cre_date' and 'Repub_date' to datetime
    def to_datetime(x):
        try:
            return pd.to_datetime(x)
        except ValueError:
            return None

    dataset['Cre_date'] = dataset['Cre_date'].apply(to_datetime)
    dataset = dataset[dataset['Cre_date'].notnull()]

    dataset['Repub_date'] = dataset['Repub_date'].apply(to_datetime)
    dataset = dataset[dataset['Repub_date'].notnull()]

    # Correct specific model values
    model_replacements = {
        "V- CLASS": 'V-Class', "E- CLASS": 'E-Class', "S- CLASS": 'S-Class',
        "Taxi": 'C-Class', "CIVIC": 'סיוויק', "JAZZ": 'ג\'אז', "ACCORD": 'אקורד',
        "CLASS": 'C-Class', "X": 'מוקה X'
    }
    dataset['model'] = dataset['model'].replace(model_replacements)

    # Function to clean and extract the model name
    def get_model(car_name):
        car_name = re.sub(r'\(.*?\)', '', car_name)
        car_name = re.sub(r'\b(החדש|חדש|חדשה|החדשה)\b', '', car_name)
        parts = car_name.split()
        if len(parts) > 1:
            if any(char.isdigit() for char in parts[1].replace(" ", "")):
                return ' '.join(parts[1:])
            else:
                model = ' '.join(parts[1:]).strip()
                model = re.sub(r'\d+', '', model).strip()
                return model if model else parts[1]
        else:
            return car_name

    dataset['model'] = dataset['model'].apply(get_model)

    # Function to remove extra spaces and characters
    def clean_car_name(car_name):
        car_name = car_name.strip()
        car_name = re.sub(r'\s?/.*', '', car_name)
        return car_name

    dataset['model'] = dataset['model'].apply(clean_car_name)
    
    #הורדת שנים חריגים
    dataset = dataset[dataset['Year'] > 1988]

    # חישוב גיל הרכב
    current_year = datetime.now().year
    dataset['Car_Age'] = current_year - dataset['Year']

    # הורדת עמודות עם הרבה נאנים או עם הרבה שגיאות
    dataset.drop(["Supply_score", "Test", "Area"], axis=1, inplace=True)

    # Convert 'Km' to numeric and fill missing values 
    dataset['Km'] = pd.to_numeric(dataset['Km'], errors='coerce').astype('Int64')
    dataset['Km'] = dataset['Km'].where(dataset['Km'] >= 1000, dataset['Km'] * 1000)
    dataset = dataset.drop(dataset[dataset['Km'] == 0].index)
    
    mean_km_by_year = dataset.groupby('Year')['Km'].median()
    dataset['Km'] = dataset.apply(lambda row: mean_km_by_year[row['Year']] if pd.isnull(row['Km']) else row['Km'], axis=1)

   
    # Ensure correct data types
    dataset['manufactor'] = dataset['manufactor'].astype(str)
    dataset['Year'] = dataset['Year'].astype(int)
    dataset['model'] = dataset['model'].astype(str)
    dataset['Hand'] = dataset['Hand'].astype(int)
    dataset['Gear'] = dataset['Gear'].astype('category')
    dataset['capacity_Engine'] = dataset['capacity_Engine'].astype(int)
    dataset['Engine_type'] = dataset['Engine_type'].astype('category')
    dataset['Prev_ownership'] = dataset['Prev_ownership'].astype('category')
    dataset['Curr_ownership'] = dataset['Curr_ownership'].astype('category')
    dataset['City'] = dataset['City'].astype(str)
    dataset['Price'] = dataset['Price'].astype(float)
    dataset['Pic_num'] = dataset['Pic_num'].astype(int)
    dataset['Description'] = dataset['Description'].astype(str)
    dataset['Color'] = dataset['Color'].astype(str)
    
    #regions
    north = [
    'חיפה', 'כרמיאל', 'ריינה', 'נהריה', "בית ג'ן", 'טירת כרמל', 'עכו', 'קרית אתא',
    'זרזיר', 'בית שאן', 'נוף הגליל', 'באקה אל גרבייה', 'באקה א שרקיה', 'יוקנעם',
    'מגדל העמק', 'עפולה', 'ראש פינה', 'באקה אל-גרביה', 'צפת', 'טירת הכרמל',
    'בוקעתא', 'אעבלין', 'טבריה', 'עין מאהל', 'מרר', 'נצרת עילית', 'זכרון יעקב',
    'קרית ביאליק', 'קריית אתא', 'נצרת', 'עתלית', 'קרית שמונה', 'משמר השרון',
    'אלמגור', 'כאבול', 'קרית טבעון', 'כפר מנדא', 'פוריידיס', 'עספיא', 'שריד',
    'אילת השחר', "סח'נין", 'כפר תבור', 'ריחאניה', 'יובלים', 'פוריה', 'אום אל פחם',
    'עראבה', 'גילון', 'תמרת', 'שפרעם', 'ערערה', 'נשר', 'דאלית אל כרמל', 'קריית ביאליק',
    'החותרים', 'אבו סנאן', 'נחף', "מג'ד אל-כרום", 'מגאר', 'מעלות תרשיחא', 'אילון',
    'חד נס', 'אבני איתן', 'ארבל', 'רכסים', 'כפר כנא', "בית ג'אן", 'קצרין', 'נווה אור',
    'גבעת אלה', 'פקיעין', 'כמון', 'מעלות', 'נהרייה', 'דבוריה', 'כסרא', 'בית קשת',
    'טמרה', 'קרית ים', 'כפר קרע', 'מזרעה', 'מצפה נטופה ד.נ. גליל תחתון', 'סאגור',
    'יקנעם עילית', 'נאעורה', 'סלמה', 'מגדל', 'יוקנעם עילית', 'חדרה',"קרית מוצקין","גבעת אבני","קריית ים","רמת מגשימים","כפר מצר"
    ]

    judea_and_samaria = [
        'עלי זהב', 'חשמונאים', 'ביתר עילית', 'מתתיהו', 'אלעזר', 'כפר תפוח',
        'מעלה אדומים', 'אריאל', 'גבע בנימין', 'בת עין', 'קרית ארבע',"רבבה"
    ]

    south = [
        'באר שבע', 'עומר', 'שדרות', 'דימונה', 'קרית גת', 'שגב שלום', 'נחלה', 'נתיבות',
        'מושב מולדת', 'אילת', 'אופקים', 'תל שבע', 'שתולים', 'כסיפה', 'ברוש', 'מעגלים',
        'קציר', 'חורה', 'מיתר', 'להבים', 'עוצם', 'אשדוד', 'אשקלון', 'קרית מלאכי',"בניה","שריגים"
    ]

    center = [
        'רעננה', 'אבן יהודה', 'רחובות', 'ראשון לציון', 'פתח תקווה', 'ירכא', 'בית דגן',
        'בת ים', 'חולון', 'גדרה', 'נס ציונה', 'גן יבנה', 'רמת גן', 'כפר סבא', 'בני ברק',
        'תל יצחק', 'תל אביב יפו', 'ראש העין', 'כפר יונה', 'גבעת שמואל', 'נתניה', 'ברקת',
        'שוהם', 'פרדס', 'גבעתיים', 'תל מונד', 'תנובות', 'אליכין', 'הוד השרון',
        'הרצליה', 'זמר', 'מכמורת', 'בית עוזיאל', 'אור עקיבא', 'אודים', 'קיסריה',
        'חרוצים', 'כפר יעבץ', 'בנימינה גבעת עדה', 'לוד', 'עזריאל',
        'מודיעין מכבים רעות', 'מודיעין', 'כפר הרי"ף', 'אלעד', 'קדרון', 'אור יהודה',
        'רמלה', 'אורנית', 'רמת השרון', 'קרית אונו', 'שער אפרים', 'זכריה',
        'פרדס חנה כרכור', 'יהוד מונוסון', 'אחיטוב', 'אזור', 'אחיעזר', 'גבעת כ"ח',
        'תל אבייב', 'יבנה', 'גבעתי', 'יהוד', 'עטרת', 'בית', 'חיננית', 'חופית', 'טייבה',
        'גני תקווה', 'יד בנימין', 'מגשימים', 'גבעתיי', 'קריות', 'ראשון', 'באר יעקב',
        'שערי תקווה', 'בארותיים', 'עמק חפר', 'פתח תיקווה', 'מזכרת בתיה', 'כפר חב"ד',
        'צפריה', 'חריש', 'רשפון', 'גן השומרון', 'מודיעין עילית', 'קרית עקרון',
        'כוכב יאיר', 'כפר מנחם', 'טירה', 'מתן', 'ניר צבי', 'קלנסווה', 'רמת ישי',
        'קדימה צורן', 'כפר עגר', 'סלעית', 'גבעת עדה', 'צור נתן', 'נגבה', 'נתנייה',
        'כפר קאסם',"צור יצחק","טייבה משולש","גבעת חיים מאוחד"
    ]

    jerusalem_and_surroundings = [
        'ירושלים', 'מבשרת ציון', 'בית שמש', 'צור הדסה', 'קרית יערים', 'אבו גוש',
        'עזריה', 'אורה', 'בית זית', 'פסגת זאב', 'גבעת זאב'
    ]

    def map_city_to_region(city):
        if city in north:
            return 'צפון'
        elif city in judea_and_samaria:
            return 'יהודה ושומרון'
        elif city in south:
            return 'דרום'
        elif city in center:
            return 'מרכז'
        elif city in jerusalem_and_surroundings:
            return 'ירושלים והסביבה'
        else:
            return 'לא ידוע'

    # בהנחה שה-DataFrame שלכם נקרא df ויש לו עמודה בשם 'City'
    dataset['Region'] = dataset['City'].apply(map_city_to_region)


    # Drop columns with too many NaNs or irrelevant columns
    dataset.drop(["Prev_ownership", "Curr_ownership"], axis=1, inplace=True)
    #ערכים לא הגיוניים לאחר שבדקנו בגרפים
    dataset=dataset[dataset['capacity_Engine']<8000]
    dataset=dataset[dataset['capacity_Engine']>150]
    dataset=dataset[dataset['Km']<500000]
    
    
    # חישוב יחס קילומטראז' לגיל הרכב
    dataset['Km_per_Year'] = dataset['Km'] / dataset['Car_Age']

    # חישוב כמה זמן עבר מאז תאריך יצירת המודעה
    dataset['Days_Since_Creation'] = (datetime.now() - dataset['Cre_date']).dt.days

    # יצירת משתנה בינארי האם פורסם מחדש או לא
    dataset['Republished'] = (dataset['Cre_date'] != dataset['Repub_date']).astype(int)
    
    # מצא ומחק כפילויות
    dataset = dataset.drop_duplicates()
    dataset.drop(['Cre_date', 'Repub_date','Year',"Description"], axis=1, inplace=True)
    
    #דירוג לפי יוקרתיות
    car_brands = {
    "מרצדס": 1,
    "ב.מ.וו": 2,
    "אאודי": 3,
    "לקסוס": 4,
    "וולוו": 5,
    "פולקסווגן": 6,
    "מיני": 7,
    "טויוטה": 8,
    "הונדה": 9,
    "מאזדה": 10,
    "סובארו": 11,
    "פורד": 12,
    "ניסאן": 13,
    "שברולט": 14,
    "קרייזלר": 15,
    "יונדאי": 16,
    "קיה": 17,
    "סקודה": 18,
    "אופל": 19,
    "פיג'ו": 20,
    "סיטרואן": 21,
    "רנו": 22,
    "מיצובישי": 23,
    "סוזוקי": 24,
    "דייהטסו": 25
    }
    dataset['company_rank'] = dataset['manufactor'].map(car_brands)
    dataset.drop(['manufactor'], axis=1, inplace=True)
    
    return dataset


In [7]:
df=prepare_data(dataset)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()
columns_to_normalize = ['Car_Age', 'Hand', 'capacity_Engine', 'Km', 'Km_per_Year', 'company_rank', 'Days_Since_Creation']
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])


#   משתנים ללא עיר,צבע,מספר תמונות ומחוז שזה עם המון פיצ'רים שלא רלוונטים
X = df[['model', 'Hand', 'Gear', 'capacity_Engine', 'Engine_type',
         'Km', 'Car_Age', 'Region',
       'Km_per_Year', 'Days_Since_Creation', 'Republished', 'company_rank']] # משתני ההסברה
X = pd.get_dummies(X, drop_first=True)
y = df['Price']  # משתנה המטרה
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Hyperparameter tuning using Grid Search 
param_grid = {
    'alpha': [0.01,0.05, 0.1, 1, 10],
    'l1_ratio': [ 0.3, 0.5, 0.7,0.85, 0.9, 0.95, 1]
}

elastic_net = ElasticNet(random_state=42)
grid_search = GridSearchCV(elastic_net, param_grid, cv=10, scoring='neg_mean_squared_error',verbose=1)
grid_search.fit(X_train, y_train)

# Best model from Grid Search
model = grid_search.best_estimator_

# הערכת ביצועי המודל
y_pred = model.predict(X_test)

# יצירת DataFrame של הנתונים המקוריים עם העמודות y_pred ו-y_test
X_test_df = pd.DataFrame(X_test, columns=X.columns)  # הפיכת X_test ל-DataFrame עם שמות העמודות המקוריים
X_test_df['y_test'] = y_test.values  # הוספת עמודת y_test
X_test_df['y_pred'] = y_pred  # הוספת עמודת y_pred

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error (RMSE) on test set: {rmse}')

# חיזוי על ידי 10-fold cross-validation
from sklearn.model_selection import cross_val_score

# Perform 10-fold cross-validation with multiple metrics
scoring = ['neg_root_mean_squared_error', 'r2']
cv_results = cross_validate(model, X_train, y_train, cv=10, scoring=scoring, return_train_score=False)

# Calculate the mean and standard deviation of RMSE and R2 scores
mean_rmse = -np.mean(cv_results['test_neg_root_mean_squared_error'])
std_rmse = np.std(cv_results['test_neg_root_mean_squared_error'])
mean_r2 = np.mean(cv_results['test_r2'])
std_r2 = np.std(cv_results['test_r2'])

print(f"Mean RMSE: {mean_rmse}")
print(f"Standard Deviation of RMSE: {std_rmse}")
print(f"Mean R2: {mean_r2}")
print(f"Standard Deviation of R2: {std_r2}")

# חפש את המאפיינים עם השפעה הגדולה ביותר על החיזוי
feature_importances = pd.Series(model.coef_, index=X_train.columns)
top_features = feature_importances.abs().nlargest(5)
print(f'Top 5 most influential features:\n{top_features}')


Fitting 10 folds for each of 35 candidates, totalling 350 fits
Root Mean Squared Error (RMSE) on test set: 9576.765330871649
Mean RMSE: 12649.874384534429
Standard Deviation of RMSE: 2017.8821818779954
Mean R2: 0.6695366735503417
Standard Deviation of R2: 0.12024822384320508
Top 5 most influential features:
Car_Age            120832.910143
model_לאונה         72899.053207
model_קורבט         69452.258227
Km                  50199.393906
capacity_Engine     44699.199044
dtype: float64


Root Mean Squared Error (RMSE) בקבוצת הבדיקה מתגבש על 9576.77, מה שמעיד על הממוצע הממוצע של השגיאה בין הערכים הצפויים לבין הערכים האמיתיים בקבוצת הבדיקה.

Mean RMSE מצביע על הערך הממוצע של RMSE במספר הקיפולים או הבדיקות השונות, כאשר הערך שלך הוא 12649.87. זה מעיד על ביצועים יציבים של המודל על מספר הנתונים השונים.

Standard Deviation of RMSE מספק פיזור של השגיאה בין הקיפולים או הבדיקות, המתפרס בטווח של 2017.88. ערכים נמוכים יותר מציינים קיבולת יעילה יותר של המודל בניתוח וניסוי של הנתונים.

Mean R2 מתצבע ב-0.67, מה שהוא סימן להיתקף טוב של המודל באופן כללי. המדד עוזר לקבוע כמה מההירות התחרות הכלליים בנתונים יכולה להיות ניספו